# Executing big data tools
The GeoAnalytics tools are presented through a set of sub modules within the `geoanalytics` module. To view the list of tools available, refer to the page titled [Working with big data](https://developers.arcgis.com/python/guide/working-with-big-data). In this page, we will learn how to execute big data tools.

## Ensuring your GIS supports GeoAnalytics
Before executing a tool, we need to ensure an ArcGIS Enterprise GIS is set up with a licensed GeoAnalytics server. To do so, call the [`is_supported()`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.geoanalytics.toc.html#is-supported) method after connecting to your Enterprise portal. See the [Components of ArcGIS URLs](http://enterprise.arcgis.com/en/portal/latest/administer/linux/components-of-arcgis-urls.htm) documentation for details on the urls to enter in the [`GIS`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html#gis) parameters based on your particular Enterprise configuration.

In [ ]:
# connect to Enterprise GIS
from arcgis.gis import GIS
import arcgis.geoanalytics

gis = GIS("your_gis_portal_url", "username", "password")

In [ ]:
# check if GeoAnalytics is supported
arcgis.geoanalytics.is_supported()

True

When no parameters are specified with `geoanalytics` methods, they use the active GIS connection, which you can query with the [`arcgis.env.active_gis`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.env.html#arcgis.env.active_gis) property.  However, if you are working with more than one GIS object, you can specify the desired GIS object as the `gis` parameter of this method. For example, let us create a connection to an Enterprise deployment and check if GeoAnalytics is supported.

In [ ]:
ago_gis = GIS("https://geoinfo_portal.esri.com/portal", "username", "password")
arcgis.geoanalytics.is_supported(ago_gis)

False

## Executing a GeoAnalytics tool
### Looking for big data file share items
When you add a big data file share, a corresponding item gets created on your portal. You can search for it like any other portal [`Item`](https://developers.arcgis.com/rest/services-reference/item.htm) and query its layers.

In [ ]:
search_result = gis.content.search("", item_type = "big data file share")
search_result

[<Item title:"bigDataFileShares_hdfs_test" type:Big Data File Share owner:admin>,
 <Item title:"bigDataFileShares_qalab" type:Big Data File Share owner:sharing3>,
 <Item title:"bigDataFileShares_Sample_US_City_Crime" type:Big Data File Share owner:admin>,
 <Item title:"bigDataFileShares_FileShareFolder" type:Big Data File Share owner:admin>]

In [ ]:
data_item = search_result[3]

<Item title:"bigDataFileShares_FileShareFolder" type:Big Data File Share owner:admin>

In [ ]:
data_item.layers

[<Layer url:"https://dev0001561.esri.com/gax/rest/services/DataStoreCatalogs/bigDataFileShares_FileShareFolder/BigDataCatalogServer/Earthquakes">,
 <Layer url:"https://dev0001561.esri.com/gax/rest/services/DataStoreCatalogs/bigDataFileShares_FileShareFolder/BigDataCatalogServer/Hurricanes">]

In [ ]:
earthquakes = data_item.layers[0]
earthquakes

<Layer url:"https://dev0001561.esri.com/gax/rest/services/DataStoreCatalogs/bigDataFileShares_FileShareFolder/BigDataCatalogServer/Earthquakes">

### Executing the Aggregate Points tool
Access the [`aggregate_points()`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.geoanalytics.summarize_data.html#aggregate-points) tool through the [`summarize_data`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.geoanalytics.summarize_data.html#) module. This example uses the Aggregate Points tool to aggregate the point features representing earthquakes into 1 Kilometer square bins. The tool creates an output feature layer in your portal you can access once processing is complete.

In [ ]:
from arcgis.geoanalytics.summarize_data import aggregate_points

The GeoAnalytics Tools use a [process spatial reference](https://enterprise.arcgis.com/en/server/latest/get-started/windows/geoanalyticstool-useenvironmentsettings.htm#ESRI_SECTION1_D37F958E528D46448D2C832D4714061C) during execution. Analyses with square or hexagon bins require a projected coordinate system. We'll use the World Cylindrical Equal Area projection (WKID 54034) below (as it is the default used when running tools in [ArcGIS Online](https://www.arcgis.com/home/index.html)). All results are stored in the spatiotemporal datastore of the Enterprise in the WGS 84 Spatial Reference.

See the GeoAnalytics Documentation for a full explanation of [analysis environment settings](https://enterprise.arcgis.com/en/server/latest/get-started/windows/geoanalyticstool-useenvironmentsettings.htm).

In [ ]:
arcgis.env.process_spatial_reference=54034

The ArcGIS Platform, including the ArcGIS API for Python, manages and transforms geographic data with a large suite of tools and functions collectively known as [geoprocessing](http://pro.arcgis.com/en/pro-app/help/analysis/geoprocessing/basics/what-is-geoprocessing-.htm). The GeoAnalytics Tools in the ArcGIS API for Python are a subset of geoprocessing tools, and operate in the context of a [geoprocessing environment](https://developers.arcgis.com/python/guide/advanced-concepts/#Geoprocessing-environment). You can set various aspects of this environment to control how tools are executed and what messages you receive during and after the execution. See the [Logging and error handling](https://developers.arcgis.com/python/guide/advanced-concepts/#Logging-and-error-handling) section in the API for Python Geoprocessing Guide [Advanced concepts](https://developers.arcgis.com/python/guide/advanced-concepts/) for ways to control messaging, including the [`arcgis.env.verbose`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.env.html#verbose) setting.

In [ ]:
import arcgis.env

arcgis.env.verbose=True

In [ ]:
agg_result = aggregate_points(earthquakes, bin_size=1, bin_size_unit='Kilometers', output_name='EQAgg_5mi_hex')

Submitted.
Executing...
Executing (AggregatePoints): AggregatePoints "Feature Set" # 1 Kilometers # # # # # # # "{"serviceProperties": {"name": "EQAgg_5mi_hex", "serviceUrl": "http://dev0001560.esri.com/server/rest/services/Hosted/EQAgg_5mi_hex/FeatureServer"}, "itemProperties": {"itemId": "ece87b3a74094a85929a2f998407e724"}}" "{"processSR": {"wkid": 54034}}"
Start Time: Thu Mar 29 16:05:24 2018
Using URL based GPRecordSet param: https://<your_gis_portal>/<web_adaptor>/rest/services/DataStoreCatalogs/bigDataFileShares_FileShareFolder/BigDataCatalogServer/Earthquakes
{"messageCode":"BD_101033","message":"'pointLayer' will be projected into the processing spatial reference.","params":{"paramName":"pointLayer"}}
{"messageCode":"BD_101028","message":"Starting new distributed job with 2 tasks.","params":{"totalTasks":"2"}}
{"messageCode":"BD_101029","message":"0/2 distributed tasks completed.","params":{"completedTasks":"0","totalTasks":"2"}}
{"messageCode":"BD_101029","message":"1/2 distri

The aggregate points tool, just like may other GeoAnalytics tools returns a feature layer item which contains the processed results.